# Material Basics

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

`Material` is the core object in Materialite. It consists of a regular 3-D grid of points and fields defined at each of those points. By default, the grid is $16\times16\times16$.

In [ ]:
from materialite import Material

material = Material()

Four attributes provide geometrical information about the `Material`:
- `dimensions`: total number of points in each direction. Default: 16 points in each direction.
- `spacing`: spacing between points in each direction. Default: 1 unit in each direction.
- `origin`: coordinates of the first point. Default: `[0, 0, 0]`
- `sizes`: total distance between the first and last point in each direction. Default: `[15, 15, 15]` (inferred from default values of the other three atributes)

In [ ]:
print(f"dimensions: {material.dimensions}")
print(f"spacing: {material.spacing}")
print(f"origin: {material.origin}")
print(f"sizes: {material.sizes}")

The fields initially consist only of geometrical information at each point. `x`, `y`, and `z` give the actual position of each point. `x_id`, `y_id`, and `z_id` are integer values giving the order of the points along each axis. Corresponding values (e.g., `x` and `x_id`) are identical when `origin = [0, 0, 0]` and `spacing = [1, 1, 1]`. Note that the points us `xyz` ordering (i.e., `z` changes fastest, then `y`, then `x`).

In [ ]:
material.get_fields()

We can also create a `Material` with specified geometrical properties.

In [ ]:
Material(dimensions=[8, 10, 12], spacing=[5, 5, 5], origin=[0, 1, 2]).get_fields()

**Creating fields**

To run models, we will typically need to add fields describing things like phase or grain ID values, material properties, etc. Here, we will look at several ways to add a field.

*create_uniform_field*

Create a uniform field (i.e., has the same value at all points). Here, set the phase ID equal to 1.

In [ ]:
material = material.create_uniform_field(label="phase", value=1)
material.get_fields()

Note that, similar to a Pandas `DataFrame`, any method that modifies a `Material` returns a new `Material`.

We can look at a particular field using the `extract` method.

In [ ]:
material.extract("phase")

*create_fields*

We can also add fields with different values at all points. Make sure that the values are ordered correctly based on the ordering of points in the fields. The new fields should be a dictionary.

In [ ]:
import numpy as np

data = np.arange(material.num_points)
new_fields = {"field1": data, "field2": data * 10}
material = material.create_fields(fields=new_fields)
print(f"field 1: {material.extract('field1')}")
print(f"field 2: {material.extract('field2')}")

Some models may require you to initialize a field of random integers.

In [ ]:
material = material.create_random_integer_field(
    label="random_integer", low=100, high=200
)
material.extract("random_integer")

Like Numpy methods that generate random numbers, the `high` value is excluded.

*create_voronoi*

We can create a Voronoi tesselation, typically to obtain an equiaxed grain structure. The tesselation can be constrained to be periodic.

In [ ]:
rng = np.random.default_rng(seed=12345)
material = material.create_voronoi(
    num_regions=10, label="grain", rng=rng, periodic=False
)
material.extract("grain")

This gives us a Voronoi tesselation of 10 grains with ID values from 0 to 9. We can also visualize the result.

In [ ]:
material.plot("grain")

*create_regional_fields*

This method allows you to assign fields according to the region that a point belongs to. Examples of regions include grains, phases, etc. Regional fields are useful for assigning things like material properties.

The syntax is similar to `create_fields`. For regional fields, we provide a dictionary (or `DataFrame`) where one key-value pair contains the unique values of the region (in this case, grains). The other key-value pair(s) are the field names and values corresponding to the region IDs. For more on regional fields, see [Regional Fields](regional_fields).

In this example, we assign each grain a stiffness equal to $100$ times the grain ID value.

In [ ]:
unique_grains = np.arange(10)
grainwise_stiffnesses = np.arange(10) * 100
regional_fields = {"grain": unique_grains, "stiffness": grainwise_stiffnesses}
material = material.create_regional_fields(
    region_label="grain", regional_fields=regional_fields
)
material.get_fields()

*assign_random_orientations*

This method assigns a random crystallographic orientation to each of the grains. A new field called "orientation" that contains a Materialite `Orientation` object (more on this in [Tensors](tensors), [Crystallography](crystallography), and [IPF Coloring](ipf_example)) is created. The `Orientation` will be identical for all points that belong to a particular grain. This is essentially a utility function for creating a particular kind of regional field.

In [ ]:
material = material.assign_random_orientations(
    region_label="grain", orientation_label="orientation"
)


material.extract("orientation")

We can also delete fields.

In [ ]:
material = material.remove_field("field1")
material.get_fields()